# Manipulating Titration Data

Next week in lab, we'll be recording and analyzing our own titration using a fully automated system that will add acid or base and record the pH as a function of volume, and spit out a datafile.  But how can we best analyze such a datafile to determine endpoints, and gain information about out sample?  This exercise will allow you to develop Python tools for this purpose.

First, let's just take a look at the datafile of pH as a function of titrant added, and make sure the data has imported correctly!  Look over, and then run, the following code block.  (Sometimes you have to run a block twice if the graph doesn't appear the first time.)



In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np


# This is how we'll import our data; it should be saved as a .csv file, in the same folder as this notebook. 
# The file must have volume in the first column and pH in the second column, with no headings on the data!

csv = np.genfromtxt ('sample_titration.csv', delimiter=",")
volume = csv[:,0]
pH = csv[:,1]

plt.plot(volume, pH, 'ro')

# Add labels on the x and y axis, always including units.
plt.xlabel('titrant added (mL)')
plt.ylabel('pH')


#let's just take a look!
plt.show()


        

In the plot command, recall that "ro" sets the symbols to red circles.  <b>See if you can change the color, and then the shape of the symbols.</b>



## Finding equivalence points

First, let's take a closer look at the data.

1. Based on the graph above, do you think is acid is a monoprotic, diprotic or triprotic acid?


2. How many equivalence points should we find?


3. Can you see them? Estimate by eye approximately where you think those equivalence points are.




It can be hard to see the equivalence points sometimes.  Remember that our pH meters can be slow to respond, and the equivalence point is where the pH is changing the fastest!  We can manipulate this data to make it a little easier to find the equivalence point. There are two strategies we can use.

1. First or second derivatives
2. Gran plot

Let's start with the derivative method!
In the code below, <b>add comments explaining what dpHdV and volume_update are</b> -- if in doubt, you can temporarily try running the code block with plt.plot using volume instead of volume_update.  Then run the original code block (with your added comments).

In [ ]:
from numpy import diff
# take the derivitive of pH with respect to volume

dpHdV = diff(pH)/diff(volume) 

volume_update = np.delete(volume, (0), axis=0) - (diff(volume) / 2)

plt.plot(volume_update, dpHdV, 'ro')

# Add labels on the x and y axis, always including units.
plt.xlabel('titrant added (mL)')
plt.ylabel('d (pH) / d(V) ')


#lets just take a look!
plt.show()

Notice that the sharp peaks are at the equivalence points! Remember, a derivitive is just a measure of how quickly your function is changing, so it is largest where the slope of the line is largest! This makes the equivalence points a lot easier to see! 
We can automatically report the volume of the highest peak, using a very simple numpy command; but notice it may return the 1st or the 2nd equivalence point, depending on which is higher, so be sure to keep them straight!

Run the code block below, then <b>modify the print statement to state whether the code found the 1st or 2nd endpoint.</b>

In [ ]:
# finding the index of the highest value in the first derivitive array
ind = np.argmax(dpHdV)

# finding the volume at the same index
endpoint_volume = volume_update[ind]

print ("the endpoint volume is " + str(endpoint_volume) + " mL")

Your textbook also discusses 2nd derivitives as an option for finding an endpoint. This can work very nicely, because the endpoints are where the data crosses zero!  <b>Add comments to the following code block explaining d2pHdV and volume_update2.</b>  Then run the block to take a look at the results:

In [ ]:
d2pHdV = diff(dpHdV)/diff(volume_update)

volume_update2 = np.delete(volume_update, (0), axis=0) - (diff(volume_update) / 2)

plt.plot(volume_update2, d2pHdV, 'r-')

# Add labels on the x and y axis, always including units.
plt.xlabel('titrant added (mL)')
plt.ylabel('d(d (pH) / d(V)) / dV ')


# Let's just take a look!
plt.show()

This is your second derivitive plot! Now our equivalence point is where the plot crosses zero. <b>To make this easier to see, change the plt.plot output from 'ro' to 'r-'.</b>  Unfortunantly, there is a lot of noise in the data, so optimizing the search for these inflection points is a harder computational problem than I would like to address here, but it can be a good reality check for the data you got from the first derivitive plot!

## Gran Plots

Our second method for finding endpoints is the Gran plot. This works by highlighting the region near the equivalence point and zeroing in on the slope of the line as it approaches the endpoint. <b> It is helpful to already have an idea where your endpoint is, roughly, before you begin this section. </b>  That's why we tried other methods first.


Gran plots can be helpful, because they take advantage of information contained in additional points leading up to the endpoint. Since the pH meter is least accurate near the endpoint, including these additional points in your calculation can help reduce some of the error coming from the sluggish kinetics of the pH meter itself.


Start by looking over the Gran_y formula below, then run the code block.

In [ ]:
# Gran plot of the entire dataset (not useable!)

Gran_y = volume * 10**(-pH)

plt.plot(volume, Gran_y, 'ro')

# Add labels on the x and y axis, always including units.
plt.xlabel('titrant added (mL)')
plt.ylabel('d(d (pH) / d(V)) / dV ')


# Let's just take a look!
plt.show()

The endpoint can be found using the steep, linear decline seen right before the graph seems to bottom out near zero. To fit this to a line, ideally we would use a linear regression on only this subset of data. 

In this case, ~ 10 to 17 mL looks about right, so let's try that by running the code block belos, and then refine our range.

<b>Add a comment to the code block identifying what "ind" is.</b>  The function "//" divides by a number and then rounds down to the nearest integer, which can then be used as an array index.


In [ ]:
# trim the array to zero in on the region between the first equivalence point, and halfway before the first equivalence points

# these can be just a starting point, and you can adjust them based on the plot
trim = ind//2
start = trim//2

Gran_trim = Gran_y[start:trim]
volume_trim = volume[start:trim]

plt.plot(volume_trim, Gran_trim, 'ro')

# Add labels on the x and y axis, always including units.
plt.xlabel('titrant added (mL)')
plt.ylabel('Vb * 10^(-pH) ')


# let's just take a look!
plt.show()

Now we want the x-intercept of this plot, which will be our first equivalence point!  However, for this to be accurate, the data that we are analyzing needs to be as straight as possible.  Look over the plot above.  Is it curved overall?  Is there a small bend at the end?  If so, we need to trim more data to zoom in on the linear, steepest part near the end.  <b>Modify the start and the trim statements so that only the linear, steepest data is showing in the graph.</b>

Once this is done, it's time to fit a straight line to the data!

Remember, if $ y= mx +b $ we solve for the x-intercept by plugging in a zero for y, and solve for x. 
Rearranging, you'll get $$ x_{intercept} = \frac{-b}{m} $$

<b>Complete and run the following code block to determine the endpoint from your Gran plot.  Then round off the number in the output statement to an appropriate number of sig figs.</b>

In [ ]:
# getting the equation of the line

import scipy.stats

# The linear regression function in the scipy stats module returns 5 values: slope, intercept, R-squared and then 
# two uncertainty values p and s_m
# We'll ignore the last two for the moment, since all we really need right now is the equation of the line
m, b, R2, p, s_m = scipy.stats.linregress(volume_trim, Gran_trim)

# solve for the x-intercept (where y = 0)

x_intercept = # complete the equation

print ("the equivalence point, determined by the Gran plot is " + str(x_intercept) + " mL") 

## Results and Analysis notes

Next week in lab, you'll use both of these methods (and these code blocks) to determine an endpoint for your tirration data, and then you'll compare the results you get on the calculated formula weight of your compound.  

<b>For now, you can save this completed Jupyter notebook with your name in the title, and upload it into your ELN!</b>